In [ ]:
import sys
sys.path.append('..')

In [ ]:
from logics_pack import global_settings, chemistry, predictor, reward_functions, ahc
from logics_pack import analysis, smiles_vocab, smiles_lstm
import pandas as pd
import numpy as np
import json
import torch

project_paths = global_settings.build_project_paths(project_dir='../')
expset_obj = global_settings.ExperimentSettings(project_paths['EXPERIMENT_SETTINGS_JSON'])

Perform AHC fine-tuning to build agent generator

In [ ]:
# AHC fine-tuning config
config = global_settings.Object()
config.tokens_path = project_paths['SMILES_TOKENS_PATH']
config.pretrain_setting_path = project_paths['PRETRAIN_SETTING_JSON']
config.pretrained_model_path = project_paths['PROJECT_DIR'] + 'model-prior/prior_e10.ckpt'
config.featurizer = predictor.featurizer
config.predictor_path = project_paths['PROJECT_DIR'] + "model-kor/predictor/kor_rfr_cv%s.pkl"%expset_obj.get_setting("kor-pred-best-cv")

config.max_epoch = 2000  # "epoch" is actually the training batches for reinforcement learning models
config.save_period = 20
config.save_size = 20000
config.save_ckpt_fmt = project_paths['PROJECT_DIR'] + 'model-kor/ahc/kor_ahc_e%d.ckpt'
config.sample_fmt = project_paths['PROJECT_DIR'] + 'model-kor/ahc/kor_ahc_e%d.txt'
config.sigma = 15
config.topk = 0.5  ## AHC
config.nbmax = 100  ## DF2
config.minscore = 0.5  ## DF2
config.dfmode = "linear"  ## DF2
config.rewarding = reward_functions.pAff_to_reward_t1
config.train_batch_size = 128
config.finetune_lr = 0.0003
config.sampling_bs = 256

config.device_name = 'cuda:5'  ####

In [ ]:
# perform fine-tuning
ahc.AHC_training(config)

Subsidiary files building for evaluation phase

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # use tensorflow cpu

import fcd
import pickle
from logics_pack import frechet_chemnet
fc_ref_model = fcd.load_ref_model()

config.vc_fmt = project_paths['PROJECT_DIR'] + 'model-kor/ahc/kor_ahc_vc_e%d.smi'  # save valid & canonical smiles
config.npfps_fmt = project_paths['PROJECT_DIR'] + 'model-kor/ahc/kor_ahc_npfps_e%d.npy'  # save fingerprint in npy
config.fcvec_fmt = project_paths['PROJECT_DIR'] + 'model-kor/ahc/kor_ahc_fcvec_e%d.npy'  # save Frechet ChemNet vectors

epochs = list(range(0, config.max_epoch+1, config.save_period))
np.array(epochs)

In [ ]:
for epo in epochs:
    print(epo)
    with open(config.sample_fmt%epo, 'r') as f:
        gens = [line.strip() for line in f.readlines()]
    vcs, invids = chemistry.get_valid_canons(gens)
    print("- count invalids: ", len(invids))
    with open(config.vc_fmt%epo, 'w') as f:
        f.writelines([line+'\n' for line in vcs])
    fps = chemistry.get_fps_from_smilist(vcs)
    np.save(config.npfps_fmt%epo, chemistry.rdk2npfps(fps))
    fcvecs = fcd.get_predictions(fc_ref_model, vcs)  # ChemNet vectors
    np.save(config.fcvec_fmt%epo, fcvecs)

Evaluate FCD and OTD on validation set, and pick the best epoch

In [ ]:
# which validation fold recorded
vfold = expset_obj.get_setting("kor-pred-best-cv")
vfold

In [ ]:
affinity_data = pd.read_csv(project_paths['KOR_DATA_PATH'])

# data split info
with open(project_paths['KOR_FOLD_JSON'], 'r') as f:
    kor_folds = json.load(f)

# retrieve validation set
val_ids = kor_folds[vfold]
val_data = affinity_data.iloc[val_ids]

# get validation set activate (vsa)
vsa_data = val_data[val_data['affinity']>global_settings.KOR_ACT_THRS]  # active among validation set
len(vsa_data)

vsa_smis = vsa_data['smiles'].tolist()
vsa_rdkfps = chemistry.get_fps_from_smilist(vsa_smis)
vsa_fc_vecs = fcd.get_predictions(fc_ref_model, vsa_smis)

dsize = len(vsa_rdkfps)  # demand size for OT
ssize = dsize*global_settings.OT_CALC_REPEATS  # supply size for repeated OT

# load predictor for PredAct (avg. predicted activity) calculation
with open(config.predictor_path, 'rb') as f:
    predictor = pickle.load(f)

In [ ]:
val_fcd_list = []
val_otd_list = []
predact_list = []

for epo in epochs:
    print(epo)
    # load fc vectors of generation
    gen_fcvecs = np.load(config.fcvec_fmt%epo)
    fcdval = frechet_chemnet.fcd_calculation(gen_fcvecs, vsa_fc_vecs)
    val_fcd_list.append(fcdval)
    
    gen_npfps = np.load(config.npfps_fmt%epo)[:ssize]  # only need this amount
    gen_rdkfps = chemistry.np2rdkfps(gen_npfps)
    simmat = analysis.calculate_simmat(gen_rdkfps, vsa_rdkfps)  # row:gen, col:data
    distmat = analysis.transport_distmat(analysis.tansim_to_dist, simmat, global_settings.OT_CALC_REPEATS)
    _, _, motds = analysis.repeated_optimal_transport(distmat, repeat=global_settings.OT_CALC_REPEATS)
    val_otd_list.append(np.mean(motds))

    # record PredAct
    predact_list.append(np.mean(predictor.predict(gen_npfps)))

In [ ]:
# validation FCDxOTD
val_FCDxOTD = np.array(val_fcd_list)*np.array(val_otd_list)
# dataframe for validation performance
v_perf = pd.DataFrame(epochs, columns=['epoch'])
v_perf['v-OTDxFCD'] = val_FCDxOTD
v_perf['v-OTD'] = val_otd_list
v_perf['v-FCD'] = val_fcd_list
v_perf['PredAct'] = predact_list
v_perf

In [ ]:
# we are only interested in epochs that achieved PredAct > (activity threshold)
subv = v_perf[v_perf['PredAct']>global_settings.KOR_ACT_THRS].copy()

# find the best epoch
vbest = subv.loc[subv['v-OTDxFCD'].idxmin()]
print(vbest)

# register the best epoch
expset_obj.update_setting('kor-ahc-best-epoch', int(vbest['epoch']))